In [1]:
# 1.csv Jan 1, 2013- April 5, 2022; 
# 2.csv Jan 1, 2003 – December 31, 2012 
# 3.csv Jan 1, 1993 - December 31, 2002 

In [2]:
import pandas as pd
import numpy as np
import math
from datetime import datetime
import pickle

In [3]:
import sqlite3
conn = sqlite3.connect('database.db')
cur = conn.cursor()

In [4]:
# use lambda fucntions for column-wise operations on every record
shortened_date = lambda d : datetime.strptime(d[:10], '%Y-%m-%d')
set_month_year = lambda d : str(d.year)+str(d.month)
set_year = lambda d : d.year
set_month = lambda d : d.month

In [5]:
ncd_cols = ['DATE', 'REPORT_TYPE', 
            'DailyAverageDryBulbTemperature', 
            'DailyAverageStationPressure',
            'DailyAverageWindSpeed', 
            'DailyCoolingDegreeDays', 'DailyDepartureFromNormalAverageTemperature', 
            'DailyHeatingDegreeDays',
            'DailyMaximumDryBulbTemperature', 'DailyMinimumDryBulbTemperature', 
            'DailyPeakWindDirection', 'DailyPeakWindSpeed', 
            'DailyPrecipitation', 
            'DailySustainedWindDirection', 'DailySustainedWindSpeed',
            'DailyWeather']

In [6]:
df1 = pd.read_csv('datasets/nenana_climate/1.csv', usecols = ncd_cols) 
df2 = pd.read_csv('datasets/nenana_climate/2.csv', usecols = ncd_cols) 
df3 = pd.read_csv('datasets/nenana_climate/3.csv', usecols = ncd_cols) 

/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3071: DtypeWarning: Columns (21,27,28,29,30,31,32,33,37,38) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,
/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3071: DtypeWarning: Columns (21,27,28,29,30,31,33,34,38,39) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [7]:
# datetime objects for easily creating other numerical fields
df1['DATE'] = df1['DATE'].apply(shortened_date)
df2['DATE'] = df2['DATE'].apply(shortened_date)
df3['DATE'] = df3['DATE'].apply(shortened_date)
# SOD = sum of day, delete all other records 
df1 = df1[df1['REPORT_TYPE']=='SOD  '].set_index('DATE').drop('REPORT_TYPE',1) 
df2 = df2[df2['REPORT_TYPE']=='SOD  '].set_index('DATE').drop('REPORT_TYPE',1) 
df3 = df3[df3['REPORT_TYPE']=='SOD  '].set_index('DATE').drop('REPORT_TYPE',1) 

In [8]:
nenana_climate = pd.concat([df1,df2,df3]).reset_index()
nenana_climate['year'] = nenana_climate['DATE'].apply(set_year)
nenana_climate['month'] = nenana_climate['DATE'].apply(set_month)

In [9]:
years_index = sorted([y for y in nenana_climate['year'].unique()])
months_index = sorted([m for m in nenana_climate['month'].unique()])

In [10]:
# Problem: these weather types are individual entries on random days 
# Solution: compact entries into monthly lists 
t= dict()
for y in years_index:
    t[y] = {}
    for m in months_index:
        t[y][m] =[]
        for v in nenana_climate[ (nenana_climate['year']==y) & (nenana_climate['month']==m) ].DailyWeather.values:
            if type(v)== str: 
                t[y][m]+= [y for y in v.split() ]

In [11]:
# to join with class labels 
weather_types = pd.DataFrame.from_dict(t);
events = pd.read_sql("""
    SELECT * 
    from nib;
    """, con=conn)

In [12]:
labels = list( events[events['Year']>=1996].month.values)
labels.append('?') # missing label most recent years 2021, 2022 
labels.append('?')
weather_types = weather_types.transpose()
weather_types['class_month'] = labels

In [13]:
# final data frame 
weather_types

,1,2,3,4,5,6,7,8,9,10,11,12,class_month
1996,"[SN, SN, FG, FG, FG, FG, FG, FG, SN, SN, SN]","[FG, FG, FG, FG, SN, FG, SN, SN, SN, SN, SN, SN]","[SN, SN, SN, SN, SN, RA]","[SN, SN, RA]","[RA, FG, RA, RA]","[RA, SN]","[RA, RA]",[],"[SN, RA, SN, SN]","[RA, SN, SN, SN, SN, SN, SN, SN, SN]","[SN, SN, SN, SN, SN, SN, SN]","[SN, SN, SN, SN, SN]",05
1997,"[BR, BR, SN, SN, BR]","[BR, SN, BR, BR, SN]","[SN, SN, SN]","[SN, SN, SN, RA]","[RA, RA, RA, RA, RA, RA]",[RA],"[RA, RA, RA]","[RA, RA, RA, RA, RA, RA, RA, RA, RA, RA, RA, R...",[],[],[],[],04
1998,"[SN, SN, SN, BR, FG, BR, HZ, SN, BR, HZ, HZ, S...","[SN, BR, HZ, RA, SN, BR, SN, BR, SN]","[SN, SN, BR, SN, BR, SN, BR, SN, SN, SN]","[SN, BR, RA, RA, SN, BR, SN, BR, SN]","[RA, SN, BR, SN, BR, RA, RA, RA, RA, BR, RA, R...","[RA, RA, RA, RA, RA, RA, RA, FG, BR, RA, FG, B...","[RA, RA, RA, BR, RA, RA, BR, RA, RA, RA, BR, R...","[RA, RA, BR, RA, RA, RA, BR, RA, BR, RA, BR, R...","[RA, RA, RA, BR, FG, BR, RA, RA, BR, FG, BR, B...","[SN, BR, SN, BR, SN, BR, FG, BR, SN, SN, BR, S...","[BR, SN, SN, SN, HZ, SN, BR, SN, BR, SN, BR, S...","[SN, BR, SN, BR, SN, BR, SN, BR, SN, BR, SN, B...",04
1999,"[SN, SN, BR, HZ, BR, SN, BR, SN, BR, SN, BR, S...","[SN, SN, HZ, SN, BR, HZ, SN, BR, HZ, SN, BR, H...","[SN, BR, SN, BR, SN, BR, HZ, HZ, SN, SN, SN, H...","[SN, BR, SN, BR, SN, FG, SN, RA, SN, BR]","[SN, BR, RA, RA, RA, FG, BR, RA, RA, RA, FG, B...","[RA, BR, RA, RA, RA, RA, RA, BR, RA, BR, RA, F...","[HZ, HZ, RA, HZ, RA, RA, RA, RA, BR, RA, BR, R...","[RA, RA, BR, RA, RA, BR, FG, BR, RA, RA, BR, R...","[RA, RA, BR, FG, BR, RA, BR, RA, RA, BR, RA, R...","[TS, FZRA, SN, BR, SN, BR, SN, BR, SN, BR, FG,...","[SN, BR, HZ, SN, BR, SN, BR, SN, BR, SN, BR, S...","[SN, SN, SN, BR, SN, SN, SN, SN, SN, SN, SN]",04
2000,[],[],[],[],[],[],[],[],[],[],[],[],05
2001,[],[],[],[],[],[],[],[],[],[],[],[],05
2002,[],[],[],[],[],[],[],[],[],[],[],[],05
2003,[],[],[],[],[],[],[],[],[],[],[],[],04
2004,"[HZ, SN, BR, SN, BR, SN, BR, HZ, BR, HZ, BR, H...","[SN, BR, SN, BR, HZ, HZ, SN, BR, SN, SN, HZ, S...","[SN, BR, HZ, SN, BR, SN, FG, BR, HZ, SN, FG, B...","[SN, SN, SN, BR, RA, RA, RA]","[RA, BR, RA, BR, RA, SN, BR, RA, BR, RA, BR, R...","[RA, RA, BR, FG, BR, RA, BR, RA, RA, RA, FG, B...","[FG, BR, HZ, BR, HZ, RA, BR, HZ, RA, BR, HZ, R...","[RA, BR, HZ, BR, HZ, BR, HZ, BR, HZ, BR, HZ, F...","[RA, BR, HZ, RA, BR, BR, BR, HZ, BR, HZ, FG, B...","[SN, SN, SN, SN, BR, SN, BR, BR, SN, BR, SN, F...","[SN, BR, SN, BR, SN, BR, SN, BR, SN, BR, SN, B...","[SN, BR, SN, BR, SN, BR, SN, BR, HZ, SN, SN, S...",04
2005,"[SN, UP, SN, BR, UP, SN, BR, UP, SN, FZFG, BR,...","[BR, HZ, BR, HZ, BR, UP, HZ, BR, HZ, BR, HZ, U...","[SN, BR, UP, SN, UP, SN, UP, UP, UP, RA, UP, U...","[SN, UP, SN, BR, UP, SN, BR, UP, SN, BR, UP, U...","[RA, HZ, RA, BR, HZ, BR, BR]","[HZ, BR, FG, BR, HZ, BR, HZ, BR, BR, BR, HZ, H...","[BR, HZ, BR, HZ, BR, BR, HZ, HZ, BR, HZ, HZ, B...","[RA, RA, BR, BR, HZ, RA, BR, HZ, RA, BR, BR, H...","[RA, UP, RA, FZFG, BR, UP, FG, RA, RA, RA, RA,...","[BR, SN, HZ, RA, SN, UP, RA, UP, SN, UP, SN, S...","[SN, FZFG, BR, UP, HZ, FG, SN, FZFG, BR, FG, U...","[SN, BR, HZ, BR, HZ, SN, SN, BR, UP, UP, UP, S...",04


In [14]:
# finally: csv file to import into classifier 
weather_types.to_csv('csv/multinomial_features.csv', index=True)
# pickle data turned out to be better to preserve the data types 
x = weather_types.transpose().to_dict()
with open('multinomial_dict.pkl', 'wb') as f:
    pickle.dump(x, f)

In [15]:
# get all weather types that exist in the data set 
existing_weather_types = set()
for i, x in zip(weather_types.index, weather_types.values): 
    for y in x[:-1]: 
        for z in y: 
            existing_weather_types.add(z)
print(existing_weather_types)

{'DZ', 'UP', 'SN', 'DU', 'HZ', 'RA', 'PL', 'BLSN', 'FZFG', 'FZRA', 'TS', 'BR', 'FG'}
